In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import lightgbm as lgb
import catboost as cb
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor

from tsfresh.examples import load_robot_execution_failures
from tsfresh import extract_features, select_features
import optuna

from common import EP

import types

/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [3]:
df_train = pd.read_pickle('feats/df_train.pkl')
df_test = pd.read_pickle('feats/df_test.pkl')

In [4]:
df_train['label'] = df_train['y'].apply(lambda x:  int(x) if x<15 else 15)
group = df_train['season'].values
group[np.where(group==17)[0]] = 1
df_train['group'] = group
df_train = df_train.drop(columns=['season'])

In [5]:
test_X = df_train.drop(columns=['y','index','group','label']).copy()
test_X.index = df_train['index']
test_y = df_train['y'].copy()
test_y.index = df_train['index']
tsfresh_columns = select_features(test_X, test_y).columns.tolist()

In [6]:
original_columns = df_train.columns.drop(['index','y','label','group']).tolist()

In [7]:
# mytrial = []
mytrial = list(pd.read_pickle('trial/extratrees.pkl').T.to_dict().values())
df_trial = pd.DataFrame(mytrial)
len(mytrial)

20

In [8]:
param={
    'algorithm': {
        'cls': 'ExtraTreesRegressor',
        'fit': {
#             'early_stopping_rounds': 200,
#             'eval_metric': 'mae',
#             'verbose': False
        },
        'init': {
            'n_jobs':16,
        }
    },
    'columns': tsfresh_columns,
    'feature_importance': {
        'is_output': True,
        'permutation_feature_importance': True,
        'permutation_random_state': 1
    },
    'kfold': {
        'n_splits': 8,
        'random_state': 1985,
        'shuffle': True,
        'type': 'group'
    },
    'scaler': {
        'cls': 'StandardScaler'
    }
}

In [9]:
# run one try
df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, param, df_test = None, trial=mytrial)

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators wil

In [11]:
#check feature_importances
df_feature_importances = df_trial.loc[0]['df_feature_importances']
if type(df_feature_importances)==pd.DataFrame:
    sorted_columns = EP.evaluate(df_feature_importances, key='average_model_weight')
else:
    sorted_columns = df_trial.loc[0]['param']['columns']
# df_feature_importances.sort_values(by=['average_permutation_weight'], ascending=False)

In [13]:
df_feature_importances.sort_values(by=['average_permutation_weight'], ascending=False).head()

,feature,model_weight,weight,std,model_weight1,weight1,std1,model_weight2,weight2,std2,...,weight5,std5,model_weight6,weight6,std6,model_weight7,weight7,std7,average_permutation_weight,average_model_weight
464,abs_q25_5,0.052660,0.050993,0.003033,0.009569,0.016907,0.002767,0.006384,0.013468,0.001875,...,0.018509,0.003787,0.039950,0.022712,0.002751,0.003110,0.011217,0.001817,0.018058,0.023446
722,iqr_6,0.009406,0.029336,0.002158,0.016296,0.016227,0.002496,0.081676,0.059811,0.003176,...,0.012627,0.000760,0.008418,0.002632,0.002199,0.001891,-0.006416,0.001320,0.013222,0.021467
868,q25_roll_std_100,0.015925,0.048078,0.003396,0.007422,0.001231,0.000597,0.008595,0.014544,0.001935,...,0.009562,0.001453,0.020279,0.015228,0.003613,0.021329,-0.067145,0.004140,0.006728,0.015305
422,Hilbert_mean_6,0.037653,0.033877,0.001231,0.001176,-0.000562,0.000618,0.001101,0.000494,0.000622,...,0.001976,0.000697,0.004455,0.014385,0.000757,0.000847,0.000326,0.000347,0.006403,0.006236
472,abs_q75_6,0.004140,0.007766,0.001074,0.049629,0.010215,0.001312,0.003830,0.011012,0.001678,...,0.004899,0.001824,0.040359,-0.004707,0.001400,0.006672,0.012425,0.002310,0.005646,0.018934


In [20]:
param = {'columns': ['q25_roll_std_100',
  'abs_q01_4',
  'abs_q25_5',
  'iqr_6',
  'mean_change_rate',
  'abs_q75_6',
  'q05_2',
  'spkt_welch_density__coeff_28',
  'abs_q75_7',
  '3th_peak_freq',
  'abs_q25_7',
  'abs_q95_2',
  'min__roll_std',
  "change_quantiles{'ql': 0.4, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
  'q01_roll_std_10',
  'iqr_8',
  'abs_q99_7',
  'spkt_welch_density__coeff_30',
  'abs_q99_8',
  'spkt_welch_density__coeff_29',
  "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
  'spkt_welch_densitycoeff_2',
  'spkt_welch_density__coeff_3',
  'abs_q25_1',
  'abs_q25_9',
  'kurt_7',
  'q95_9',
  'spkt_welch_density__coeff_42',
  "autocorrelation{'lag': 5}",
  'abs_q05_9',
  'fft_coefficientcoeff_26__attr_"abs"',
  'q25_1',
  'abs_q25_8',
  'fft_coefficientcoeff_20__attr_"abs"',
  'abs_q01_7',
  'partial_autocorrelationlag_1',
  'iqr_9',
  'spkt_welch_density__coeff_65',
  'spkt_welch_density__coeff_50',
  '5000skewness_mean_',
  'fft_coefficientcoeff_16__attr_"imag"',
  'FFT_Mag_75q0',
  'med_4',
  'spkt_welch_density__coeff_117',
  'spkt_welch_density__coeff_41',
  'spkt_welch_density__coeff_38',
  'abs_q95_8',
  'partial_autocorrelationlag_5',
  'peak_to_average_power_ratio__roll_mean',
  'spkt_welch_density__coeff_63'],
 'kfold': {'n_splits': 8,
  'random_state': 1985,
  'shuffle': True,
  'type': 'stratified'},
 'scaler': {'cls': 'StandardScaler'},
 'algorithm': {'cls': 'ExtraTreesRegressor',
  'init': {'n_estimators': 362,
   'max_depth': 8,
   'max_features': 0.9444476808681541,
   'min_samples_leaf': 0.10100912696889522,
   'random_state': 7503},
  'fit': {}},
 'feature_importance': {'is_output': True,
  'permutation_feature_importance': True,
  'permutation_random_state': 1}}

In [21]:
#  select features by permutation_weight
EP.select_features_(df_train, param, mytrial, nfeats_best=25, nfeats_removed_per_try=5, key='average_permutation_weight')

In [24]:
# [(df_trial['mae_diff']<.05)].sort_values(by=['val_mae'], ascending=True)
df_trial = pd.DataFrame(mytrial)
df_trial[['datetime','remark','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
0,2019-05-10 08:42:07.651275,None,1071,1.499165e-06,1.468636e-14,2.269274,0.489602,2.269272
1,2019-05-10 09:02:37.071088,None,200,1.197445e-06,9.774471e-15,2.260614,0.457957,2.260612
2,2019-05-10 09:16:09.476415,None,190,1.155385e-06,1.348192e-14,2.255028,0.468715,2.255027
3,2019-05-10 09:28:58.440117,None,180,1.146699e-06,5.994585e-15,2.240094,0.479207,2.240093
4,2019-05-10 09:41:04.616695,None,170,1.162662e-06,6.818891e-15,2.225713,0.467771,2.225712
5,2019-05-10 09:52:28.280265,None,160,1.145904e-06,1.329546e-14,2.214935,0.484027,2.214934
6,2019-05-10 10:03:09.978947,None,150,1.146723e-06,1.277012e-14,2.209481,0.474300,2.209479
7,2019-05-10 10:13:08.307155,None,140,1.156019e-06,1.259827e-14,2.203670,0.460615,2.203669
8,2019-05-10 10:22:23.215791,None,130,1.120365e-06,7.589212e-15,2.204208,0.458124,2.204207
9,2019-05-10 10:30:54.595849,None,120,1.104875e-06,1.124369e-14,2.209440,0.455868,2.209439


In [23]:
#  tune hypterparameters
def objective(trial):
        
    n_estimators = trial.suggest_int('n_estimators', 300, 1000)
    max_depth = trial.suggest_int('max_depth', 5, 16)
    max_features = trial.suggest_uniform('max_features', .6, 1)
    min_samples_leaf = trial.suggest_uniform('min_samples_leaf', 0.1, 0.5)
    random_state = trial.suggest_int('random_state', 1, 9999)
        
    args={
        'columns':df_trial.loc[14]['param']['columns'],
        'kfold':{
            'n_splits': 8,
            'random_state': 1985,
            'shuffle': True,
            'type': 'stratified'
        },
        'scaler':{
            'cls':'StandardScaler',
        },
        'algorithm':{
            'cls':'ExtraTreesRegressor',
            'init':{
                "n_estimators":n_estimators,
                "max_depth":max_depth,
                "max_features":max_features,
                "min_samples_leaf":min_samples_leaf,
                "random_state":random_state,
            },
            'fit':{
#                 'eval_metric':'mae',
#                 'verbose':False,
#                 'early_stopping_rounds':200,
            },
        },
        'feature_importance':{
            'is_output':False,
            'permutation_feature_importance':False,
            'permutation_random_state':1,
        },
    }
    
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='tune feats selected by group2')
    val_mae_mean = np.mean(df_his.valid)
    val_mae_var = np.var(df_his.valid)
    train_mae_mean = np.mean(df_his.train)
    train_mae_var = np.var(df_his.train)
    
    trial.set_user_attr('val_mae', val_mae_mean)
    trial.set_user_attr('train_mae', train_mae_mean)
    trial.set_user_attr('mae_diff', val_mae_mean-train_mae_mean)
    trial.set_user_attr('val_mae_var', val_mae_var)

    return np.abs(val_mae_mean - train_mae_mean)*val_mae_mean

study = optuna.create_study()
study.optimize(objective, n_trials=200)

[I 2019-05-11 17:10:51,533] Finished trial#0 resulted in value: 9.654926805258963e-05. Current best value is 9.654926805258963e-05 with parameters: {'n_estimators': 352, 'max_depth': 11, 'max_features': 0.7333850687472897, 'min_samples_leaf': 0.38720508306365553, 'random_state': 3281}.
[I 2019-05-11 17:14:52,120] Finished trial#1 resulted in value: 0.0006920156616231911. Current best value is 9.654926805258963e-05 with parameters: {'n_estimators': 352, 'max_depth': 11, 'max_features': 0.7333850687472897, 'min_samples_leaf': 0.38720508306365553, 'random_state': 3281}.
[I 2019-05-11 17:18:16,236] Finished trial#2 resulted in value: 0.001144608143627626. Current best value is 9.654926805258963e-05 with parameters: {'n_estimators': 352, 'max_depth': 11, 'max_features': 0.7333850687472897, 'min_samples_leaf': 0.38720508306365553, 'random_state': 3281}.
[I 2019-05-11 17:19:56,535] Finished trial#3 resulted in value: 0.0005748649831315537. Current best value is 9.654926805258963e-05 with para

[I 2019-05-11 18:14:07,709] Finished trial#28 resulted in value: 0.00013825994405622727. Current best value is 1.1463573298955164e-05 with parameters: {'n_estimators': 689, 'max_depth': 14, 'max_features': 0.9722171219512438, 'min_samples_leaf': 0.4956032788566416, 'random_state': 4648}.
[I 2019-05-11 18:15:20,489] Finished trial#29 resulted in value: 0.0004739616787304101. Current best value is 1.1463573298955164e-05 with parameters: {'n_estimators': 689, 'max_depth': 14, 'max_features': 0.9722171219512438, 'min_samples_leaf': 0.4956032788566416, 'random_state': 4648}.
[I 2019-05-11 18:16:37,388] Finished trial#30 resulted in value: 0.0003030128615385858. Current best value is 1.1463573298955164e-05 with parameters: {'n_estimators': 689, 'max_depth': 14, 'max_features': 0.9722171219512438, 'min_samples_leaf': 0.4956032788566416, 'random_state': 4648}.
[I 2019-05-11 18:20:30,630] Finished trial#31 resulted in value: 0.0015991662694365874. Current best value is 1.1463573298955164e-05 wi

[I 2019-05-11 18:52:47,469] Finished trial#56 resulted in value: 0.001151232511142083. Current best value is 4.9467106572475e-06 with parameters: {'n_estimators': 312, 'max_depth': 13, 'max_features': 0.6019196543102411, 'min_samples_leaf': 0.4980700590919195, 'random_state': 6269}.
[I 2019-05-11 18:56:42,138] Finished trial#57 resulted in value: 0.0011207302441184578. Current best value is 4.9467106572475e-06 with parameters: {'n_estimators': 312, 'max_depth': 13, 'max_features': 0.6019196543102411, 'min_samples_leaf': 0.4980700590919195, 'random_state': 6269}.
[I 2019-05-11 18:57:50,347] Finished trial#58 resulted in value: 0.00018115336652515743. Current best value is 4.9467106572475e-06 with parameters: {'n_estimators': 312, 'max_depth': 13, 'max_features': 0.6019196543102411, 'min_samples_leaf': 0.4980700590919195, 'random_state': 6269}.
[I 2019-05-11 18:59:50,528] Finished trial#59 resulted in value: 0.0004105746045095572. Current best value is 4.9467106572475e-06 with parameters

[I 2019-05-11 19:37:38,868] Finished trial#84 resulted in value: 0.00011829800451324949. Current best value is 4.9467106572475e-06 with parameters: {'n_estimators': 312, 'max_depth': 13, 'max_features': 0.6019196543102411, 'min_samples_leaf': 0.4980700590919195, 'random_state': 6269}.
[I 2019-05-11 19:38:30,140] Finished trial#85 resulted in value: 0.00022008847377986493. Current best value is 4.9467106572475e-06 with parameters: {'n_estimators': 312, 'max_depth': 13, 'max_features': 0.6019196543102411, 'min_samples_leaf': 0.4980700590919195, 'random_state': 6269}.
[I 2019-05-11 19:39:23,351] Finished trial#86 resulted in value: 0.00020520782232077243. Current best value is 4.9467106572475e-06 with parameters: {'n_estimators': 312, 'max_depth': 13, 'max_features': 0.6019196543102411, 'min_samples_leaf': 0.4980700590919195, 'random_state': 6269}.
[I 2019-05-11 19:40:11,858] Finished trial#87 resulted in value: 5.680733967627178e-05. Current best value is 4.9467106572475e-06 with paramet

[I 2019-05-11 20:33:44,160] Finished trial#112 resulted in value: 0.0009037043476022027. Current best value is 4.9467106572475e-06 with parameters: {'n_estimators': 312, 'max_depth': 13, 'max_features': 0.6019196543102411, 'min_samples_leaf': 0.4980700590919195, 'random_state': 6269}.
[I 2019-05-11 20:34:55,117] Finished trial#113 resulted in value: 0.0005169853767601783. Current best value is 4.9467106572475e-06 with parameters: {'n_estimators': 312, 'max_depth': 13, 'max_features': 0.6019196543102411, 'min_samples_leaf': 0.4980700590919195, 'random_state': 6269}.
[I 2019-05-11 20:36:35,758] Finished trial#114 resulted in value: 7.565872188858489e-05. Current best value is 4.9467106572475e-06 with parameters: {'n_estimators': 312, 'max_depth': 13, 'max_features': 0.6019196543102411, 'min_samples_leaf': 0.4980700590919195, 'random_state': 6269}.
[I 2019-05-11 20:38:17,761] Finished trial#115 resulted in value: 0.00015285934560489925. Current best value is 4.9467106572475e-06 with param

[I 2019-05-11 21:21:33,853] Finished trial#140 resulted in value: 0.0008535074192114446. Current best value is 4.9467106572475e-06 with parameters: {'n_estimators': 312, 'max_depth': 13, 'max_features': 0.6019196543102411, 'min_samples_leaf': 0.4980700590919195, 'random_state': 6269}.
[I 2019-05-11 21:23:07,202] Finished trial#141 resulted in value: 9.620072755323793e-05. Current best value is 4.9467106572475e-06 with parameters: {'n_estimators': 312, 'max_depth': 13, 'max_features': 0.6019196543102411, 'min_samples_leaf': 0.4980700590919195, 'random_state': 6269}.
[I 2019-05-11 21:24:31,645] Finished trial#142 resulted in value: 0.0003484884399509277. Current best value is 4.9467106572475e-06 with parameters: {'n_estimators': 312, 'max_depth': 13, 'max_features': 0.6019196543102411, 'min_samples_leaf': 0.4980700590919195, 'random_state': 6269}.
[I 2019-05-11 21:25:51,462] Finished trial#143 resulted in value: 0.00011040336775032124. Current best value is 4.9467106572475e-06 with param

[I 2019-05-11 22:17:28,687] Finished trial#168 resulted in value: 1.0385642844682012e-05. Current best value is 4.9467106572475e-06 with parameters: {'n_estimators': 312, 'max_depth': 13, 'max_features': 0.6019196543102411, 'min_samples_leaf': 0.4980700590919195, 'random_state': 6269}.
[I 2019-05-11 22:20:05,487] Finished trial#169 resulted in value: 6.0224116267889655e-05. Current best value is 4.9467106572475e-06 with parameters: {'n_estimators': 312, 'max_depth': 13, 'max_features': 0.6019196543102411, 'min_samples_leaf': 0.4980700590919195, 'random_state': 6269}.
[I 2019-05-11 22:23:34,319] Finished trial#170 resulted in value: 0.0007677202371336787. Current best value is 4.9467106572475e-06 with parameters: {'n_estimators': 312, 'max_depth': 13, 'max_features': 0.6019196543102411, 'min_samples_leaf': 0.4980700590919195, 'random_state': 6269}.
[I 2019-05-11 22:27:18,656] Finished trial#171 resulted in value: 0.0008997126920355226. Current best value is 4.9467106572475e-06 with para

[I 2019-05-11 23:22:36,501] Finished trial#196 resulted in value: 1.975912792667331e-05. Current best value is 4.9467106572475e-06 with parameters: {'n_estimators': 312, 'max_depth': 13, 'max_features': 0.6019196543102411, 'min_samples_leaf': 0.4980700590919195, 'random_state': 6269}.
[I 2019-05-11 23:26:19,448] Finished trial#197 resulted in value: 0.0013284111002907037. Current best value is 4.9467106572475e-06 with parameters: {'n_estimators': 312, 'max_depth': 13, 'max_features': 0.6019196543102411, 'min_samples_leaf': 0.4980700590919195, 'random_state': 6269}.
[I 2019-05-11 23:28:31,722] Finished trial#198 resulted in value: 0.000159987891592169. Current best value is 4.9467106572475e-06 with parameters: {'n_estimators': 312, 'max_depth': 13, 'max_features': 0.6019196543102411, 'min_samples_leaf': 0.4980700590919195, 'random_state': 6269}.
[I 2019-05-11 23:31:02,168] Finished trial#199 resulted in value: 0.0003140747429036838. Current best value is 4.9467106572475e-06 with paramet

In [43]:
df_test_pred = df_trial.loc[342]['df_test_pred']

In [44]:
df_submit = pd.DataFrame()
df_submit['time_to_failure'] = np.mean(df_test_pred.drop(columns=['index']).values, axis=1)
df_submit['seg_id'] = df_test_pred['index']
df_submit.to_csv('submission.csv', index=False)

In [14]:
df_trial = pd.DataFrame(mytrial)

In [25]:
#[df_trial['mae_diff']<.05].sort_values(by=['val_mae']) | (df_trial['remark']=='tune feats selected by group ')
df_trial[(df_trial['remark']=='tune feats selected by group2')&(df_trial['mae_diff']<.05)].sort_values(by=['val_mae'], ascending=True)[['datetime','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']].head(10)

,datetime,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
250,2019-05-11 18:05:23.692238,70,2.107760,0.000011,2.108616,0.000522,0.000856
391,2019-05-11 22:09:12.488601,70,2.125332,0.000017,2.125989,0.000466,0.000657
258,2019-05-11 18:20:30.570967,70,2.126089,0.000017,2.126841,0.000479,0.000752
283,2019-05-11 18:52:47.407807,70,2.134329,0.000026,2.134868,0.000414,0.000539
322,2019-05-11 19:57:14.231200,70,2.142904,0.000022,2.143377,0.000435,0.000473
424,2019-05-11 23:26:19.336513,70,2.151488,0.000021,2.152106,0.000422,0.000617
413,2019-05-11 23:02:44.157146,70,2.160694,0.000016,2.161204,0.000421,0.000510
269,2019-05-11 18:34:59.566361,70,2.171977,0.000019,2.172202,0.000402,0.000225
233,2019-05-11 17:29:48.912459,70,2.173353,0.000023,2.173832,0.000390,0.000480
328,2019-05-11 20:07:48.374718,70,2.174271,0.000012,2.174880,0.000445,0.000609


In [26]:
df_trial.loc[250:250][['datetime','remark','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
250,2019-05-11 18:05:23.692238,tune feats selected by group2,70,2.10776,0.000011,2.108616,0.000522,0.000856


In [27]:
df_trial.to_pickle('trial/extratrees.pkl')

In [29]:
df_trial.loc[250]['param']

{'columns': ['q25_roll_std_100',
  'abs_q25_5',
  'abs_q01_4',
  'iqr_6',
  'abs_q75_6',
  'mean_change_rate',
  'abs_q25_7',
  'abs_q75_7',
  'q05_2',
  'q01_roll_std_10',
  "change_quantiles{'ql': 0.4, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
  'q25_1',
  'spkt_welch_density__coeff_29',
  '3th_peak_freq',
  'spkt_welch_densitycoeff_2',
  'abs_q95_2',
  'abs_q25_9',
  'abs_q25_1',
  'min__roll_std',
  'spkt_welch_density__coeff_30',
  'spkt_welch_density__coeff_28',
  'iqr_8',
  'abs_q75_1',
  'abs_q95_8',
  'abs_max_8',
  'kurt_7',
  "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
  'fft_coefficientcoeff_26__attr_"abs"',
  'abs_max_7',
  'fft_coefficientcoeff_21__attr_"abs"',
  'q95_9',
  'spkt_welch_density__coeff_3',
  'abs_q01_7',
  'abs_q99_8',
  'abs_q99_7',
  'fft_coefficientcoeff_32__attr_"imag"',
  'peak_to_average_power_ratio__roll_mean',
  'Hilbert_mean_1',
  'iqr_9',
  'fft_coefficientcoeff_20__attr_"abs"',
  'min_roll_std_100',
  'FFT_Mag_75q